# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/).

## Import your imports

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

## Search for the TLDR Number `006179570C`, and scrape the information on that company

Using [license information system](https://www.tdlr.texas.gov/tools_search/), find information about the tow truck number above, displaying the

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

If you can't figure a 'nice' way to locate something, your two last options might be:

- **Find a "parent" element, then dig inside**
- **Find all of a type of element** (like we did with `td` before) and get the `[0]`, `[1]`, `[2]`, etc
- **XPath** (inspect an element, Copy > Copy XPath)

These kinds of techniques tend to break when you're on other result pages, but... maybe not! You won't know until you try.

> - *TIP: When you use xpath, you CANNOT use double quotes or Python will get confused. Use single quotes.*
> - *TIP: You can clean your data up if you want to, or leave it dirty to clean later*
> - *TIP: The address part can be tough, but you have a few options. You can use a combination of `.split` and list slicing to clean it now, or clean it later in the dataframe with regular expressions. Or other options, too, probably*

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get('https://www.tdlr.texas.gov/tools_search/')

In [5]:
driver.find_element_by_id('mcrdata').send_keys('006179570C')

In [8]:
button = driver.find_element_by_id('submit3')
button.click()

In [104]:
comp_name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text[8:]
comp_name

'BARRY MICHAEL SMITH'

In [110]:
owner = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text[17:]
owner

'BARRY MICHAEL SMITH / OWNER'

In [112]:
phone = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text[9:]
phone


'8066544404'

In [111]:
len('Phone:   ')

9

In [114]:
lic_status = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[1]/font').text[11:]
lic_status

'(Active)'

In [118]:
address = (" ").join(driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text.split("\n")[-2:])
address




'4501 W CEMETERY RD CANYON, TX. 79015'

In [46]:
#    The business name
#    Owner/operator
#    Phone number
#    License status (Active, Expired, Etc)
#    Physical address


# Adapt this to work inside of a single cell

Double-check that it works. You want it to print out all of the details.

In [119]:
comp_name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text[8:]
print(comp_name)
owner = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text[17:]
print(owner)
phone = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text[9:]
print(phone)
lic_status = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[1]/font').text[11:]
print(lic_status)
address = (" ").join(driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text.split("\n")[-2:])
print(address)

BARRY MICHAEL SMITH
BARRY MICHAEL SMITH / OWNER
8066544404
(Active)
4501 W CEMETERY RD CANYON, TX. 79015


# Using .apply to find data about SEVERAL tow truck companies

The file `trucks-subset.csv` has information about the trucks, we'll use it to find the pages to scrape.

### Open up `trucks-subset.csv` and save it into a dataframe

In [53]:
import pandas as pd
df = pd.read_csv("trucks-subset.csv")
df.head()

,TDLR Number
0,006507931C
1,006179570C
2,006502097C


## Go through each row of the dataset, displaying the URL you will need to scrape for the information on that row

You don't have to actually use the search form for each of these - look at the URL you're on, it has the number in it!

For example, one URL might look like `https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C`.

- *TIP: Use .apply and a function*
- *TIP: Unlike the Yelp example, you'll need to build this URL from pieces*
- *TIP: You probably don't want to `print` unless you're going to fix it for the next question 
- *TIP: pandas won't showing you the entire url! Run `pd.set_option('display.max_colwidth', -1)` to display aaaalll of the text in a cell*

In [55]:
pd.set_option('display.max_colwidth', -1)

In [76]:
def get_url(TDLRNumber):
    return 'https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber='+TDLRNumber

### Save this URL into a new column of your dataframe, called `url`

- *TIP: Use a function and `.apply`*
- *TIP: Be sure to use `return`*

In [77]:
df['url'] = df['TDLR Number'].apply(get_url)
df.head()

,TDLR Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Go through each row of the dataset, printing out information about each tow truck company.

Now will be **scraping** inside of your function.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

Just print it out for now.

- *TIP: use .apply*
- *TIP: You'll be using the code you wrote before, but converted into a function*
- *TIP: Remember how the TDLR Number is in the URL? You don't need to do the form submission if you don't want!*
- *TIP: Make sure you adjust any variables so you don't scrape the same page again and again*

In [148]:
def get_company_info(row):
    driver.get(row['url'])

    comp_name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text[8:]

    owner = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text[17:]
   
    phone = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text[9:]
    
    lic_status = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[1]/font').text[11:]
    
    address = (" ").join(driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text.split("\n")[-2:])
    
    print (comp_name)
    print (owner)
    print (phone)
    print (lic_status)
    print (address)
    print ("******")



In [152]:
df.apply(get_company_info, axis=1)


AUGUSTUS E SMITH
AUGUSTUS EUGENE SMITH / OWNER
9032276464
(Active)
103 N MAIN ST BONHAM, TX. 75418
******
B.D. SMITH TOWING
BRANDT SMITH / OWNER
8173330706
(Expired)
13619 BRETT JACKSON RD. FORT WORTH, TX. 76179
******
BARRY MICHAEL SMITH
BARRY MICHAEL SMITH / OWNER
8066544404
(Active)
4501 W CEMETERY RD CANYON, TX. 79015
******


0    None
1    None
2    None
dtype: object

## Scrape the following information for each row of the dataset, and save it into new columns in your dataframe.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

It's basically what we did before, but using the function a little differently.

- *TIP: Same as above, but you'll be returning a `pd.Series` and the `.apply` line is going to be a lot longer*
- *TIP: Save it to a new dataframe!*
- *TIP: Make sure you change your `df` variable names correctly if you're cutting and pasting - there are a few so it can get tricky*

In [153]:
def get_company_info(row):
    driver.get(row['url'])

    comp_name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text[8:]

    owner = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text[17:]
   
    phone = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text[9:]
    
    lic_status = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[1]/font').text[11:]
    
    address = (" ").join(driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text.split("\n")[-2:])
    
    return  pd.Series ({
        'comp_name': comp_name,
        'owner': owner,
        'phone': phone,
        'lic_status': lic_status,
        'address': address
       })

In [169]:
new_df = df.apply(get_company_info, axis=1).join(df)
new_df

,comp_name,owner,phone,lic_status,address,TDLR Number,url
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,(Active),"103 N MAIN ST BONHAM, TX. 75418",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,(Expired),"13619 BRETT JACKSON RD. FORT WORTH, TX. 76179",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,(Active),"4501 W CEMETERY RD CANYON, TX. 79015",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


### Save your dataframe as a CSV named `tow-trucks-extended.csv`

In [170]:
new_df.to_csv("tow-trucks-extended.csv", index=False)

### Re-open your dataframe to confirm you didn't save any extra weird columns

In [160]:
new_df = pd.read_csv("tow-trucks-extended.csv")
new_df.head()

,comp_name,owner,phone,lic_status,address,TDLR Number,url
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,(Active),"103 N MAIN ST BONHAM, TX. 75418",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,(Expired),"13619 BRETT JACKSON RD. FORT WORTH, TX. 76179",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,(Active),"4501 W CEMETERY RD CANYON, TX. 79015",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Process the entire `tow-trucks.csv` file

We just did it on a short subset so far. Now try it on all of the tow trucks. **Save as the same filename as before**

In [161]:
df2 = pd.read_csv("tow-trucks.csv")

In [162]:
df2.head()

,TDLR Number
0,006507931C
1,006179570C
2,006502097C
3,006494912C
4,0649468VSF


In [163]:
def get_url(TDLRNumber):
    return 'https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber='+TDLRNumber

In [165]:
df2['url'] = df2['TDLR Number'].apply(get_url)
df2.head()

,TDLR Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
3,006494912C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C
4,0649468VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF
5,006448786C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006448786C
6,0648444VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0648444VSF
7,0651667VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0651667VSF
8,006017767C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006017767C
9,006495492C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C


In [166]:
def get_company_info(row):
    driver.get(row['url'])

    comp_name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text[8:]

    owner = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text[17:]
   
    phone = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text[9:]
    
    lic_status = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[1]/font').text[11:]
    
    address = (" ").join(driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text.split("\n")[-2:])
    
    return  pd.Series ({
        'comp_name': comp_name,
        'owner': owner,
        'phone': phone,
        'lic_status': lic_status,
        'address': address
       })

In [172]:
new_df2 = df2.apply(get_company_info, axis=1).join(df2)
new_df2.head(10)

,comp_name,owner,phone,lic_status,address,TDLR Number,url
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,(Active),"103 N MAIN ST BONHAM, TX. 75418",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,(Expired),"13619 BRETT JACKSON RD. FORT WORTH, TX. 76179",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,(Active),"4501 W CEMETERY RD CANYON, TX. 79015",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
3,HEATH SMITH,HEATH A SMITH / OWNER,940-552-0687,(Expired),"1529 WILBARGER ST VERNON, TX. 76384",006494912C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C
4,HEATH SMITH,HEATH A SMITH / OWNER,9405520687,(Expired),"1529 WILBARGER ST VERNON, TX. 76384",0649468VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF
5,HYSMITH AUTOMOTIVE,WILLIAM THOMAS HYSMITH / PRESIDENT,icer: ASHLEY ERIN HYSMITH / TREASURER,(Active),"1210 US 380 BYPASS GRAHAM, TX. 76450",006448786C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006448786C
6,HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,HYSMITH ERIN ASHLEY / TREASURER,icer: WILLIAM THOMAS HYSMITH / PRESIDENT,(Suspended),"927 LOVING HWY GRAHAM, TX. 76450",0648444VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0648444VSF
7,HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,WILLIAM THOMAS HYSMITH / PRESIDENT,icer: ASHLEY ERIN HYSMITH / TREASURER,(Active),"1210 380 BYPASS GRAHAM, TX. 76450",0651667VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0651667VSF
8,JEFF & WENDY SMITH,JEFF SMITH / PARTNER,icer: WENDY SMITH / PARTNER,(Suspended),"10842 FM 2138 N JACKSONVILLE, TX. 75766",006017767C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006017767C
9,JEFF SMITH,JEFFREY JOHN SMITH / OWNER,8324354670,(Active),"4338 HARVEY RD CROSBY, TX. 77532",006495492C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C


In [173]:
new_df2.to_csv("tow-trucks-extended.csv", index=False)

In [176]:
df3 = pd.read_csv("tow-trucks-extended.csv")
df3.head(10)

,comp_name,owner,phone,lic_status,address,TDLR Number,url
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,(Active),"103 N MAIN ST BONHAM, TX. 75418",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,(Expired),"13619 BRETT JACKSON RD. FORT WORTH, TX. 76179",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,(Active),"4501 W CEMETERY RD CANYON, TX. 79015",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
3,HEATH SMITH,HEATH A SMITH / OWNER,940-552-0687,(Expired),"1529 WILBARGER ST VERNON, TX. 76384",006494912C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C
4,HEATH SMITH,HEATH A SMITH / OWNER,9405520687,(Expired),"1529 WILBARGER ST VERNON, TX. 76384",0649468VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF
5,HYSMITH AUTOMOTIVE,WILLIAM THOMAS HYSMITH / PRESIDENT,icer: ASHLEY ERIN HYSMITH / TREASURER,(Active),"1210 US 380 BYPASS GRAHAM, TX. 76450",006448786C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006448786C
6,HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,HYSMITH ERIN ASHLEY / TREASURER,icer: WILLIAM THOMAS HYSMITH / PRESIDENT,(Suspended),"927 LOVING HWY GRAHAM, TX. 76450",0648444VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0648444VSF
7,HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,WILLIAM THOMAS HYSMITH / PRESIDENT,icer: ASHLEY ERIN HYSMITH / TREASURER,(Active),"1210 380 BYPASS GRAHAM, TX. 76450",0651667VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0651667VSF
8,JEFF & WENDY SMITH,JEFF SMITH / PARTNER,icer: WENDY SMITH / PARTNER,(Suspended),"10842 FM 2138 N JACKSONVILLE, TX. 75766",006017767C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006017767C
9,JEFF SMITH,JEFFREY JOHN SMITH / OWNER,8324354670,(Active),"4338 HARVEY RD CROSBY, TX. 77532",006495492C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C
